In [1]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

In [2]:
const { execSync } = await import('child_process');
console.log(execSync('npm install @viz-js/viz').toString());
import { RegExp2NFA, parseRegex, EPS, showNFA, nfa2dot, NFA, nfa2dfa, dfaToDot, minimizeDFA}  from "./03-Regexp-2-NFA.js";
import * as fs from "fs";
import { instance } from "@viz-js/viz";


up to date, audited 13 packages in 854ms

1 package is looking for funding
  run `npm fund` for details

found 0 vulnerabilities



This notebook contains a test for the function `toNFA`.

If the regular expression `r` that is defined below is written in the style of the lecture notes, it reads:
$$(\texttt{a}\cdot\texttt{b} + \texttt{b}\cdot\texttt{a})^*$$

In [3]:
const r = parseRegex("(a⋅b+b⋅a)*");
console.log(JSON.stringify(r, null, 2));

{
  "kind": "star",
  "expr": {
    "kind": "union",
    "left": {
      "kind": "concat",
      "left": {
        "kind": "sym",
        "ch": "a"
      },
      "right": {
        "kind": "sym",
        "ch": "b"
      }
    },
    "right": {
      "kind": "concat",
      "left": {
        "kind": "sym",
        "ch": "b"
      },
      "right": {
        "kind": "sym",
        "ch": "a"
      }
    }
  }
}


We use `converter` to create a non-deterministic <span style="font-variant:small-caps;">Fsm</span> `nfa` that accepts the language 
described by the regular expression `r`.

In [4]:
const Sigma = new Set(["a", "b"]);
const converter = new RegExp2NFA(Sigma);
const nfa = converter.toNFA(r);

console.log("Constructed NFA:");
console.log(showNFA(nfa));

Constructed NFA:
({11, 12, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8}, {a, b}, {(1, 'a'): {2}, (3, 'b'): {4}, (2, 'ε'): {3}, (5, 'b'): {6}, (7, 'a'): {8}, (6, 'ε'): {7}, (9, 'ε'): {1, 5}, (4, 'ε'): {10}, (8, 'ε'): {10}, (11, 'ε'): {9, 12}, (10, 'ε'): {9, 12}}, 11, {12})


In [5]:
async function showNFA(nfa: NFA) {
  const viz = await instance();
  const dot = nfa2dot(nfa);
  const svg = await viz.renderString(dot, { format: "svg" });

  // Wenn du in Jupyter / VS Code Notebook arbeitest:
  if (typeof display !== "undefined" && display.html) {
    display.html(svg);
  } else {
    // fallback in der Konsole
    console.log(svg);
  }
}

await showNFA(nfa);

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.1 (20251006.0113)
 -->
<!-- Title: NFA Pages: 1 -->
 
 
 NFA 
 
<!-- __start -->
 
 __start 
 
 
<!-- 11 -->
 
 11 
 
 11 
 
<!-- __start->11 -->
 
 __start->11 
 
 
 
<!-- 12 -->
 
 12 
 
 
 12 
 
<!-- 11->12 -->
 
 11->12 
 
 
 ε 
 
<!-- 9 -->
 
 9 
 
 9 
 
<!-- 11->9 -->
 
 11->9 
 
 
 ε 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 1->2 -->
 
 1->2 
 
 
 a 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 2->3 -->
 
 2->3 
 
 
 ε 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 3->4 -->
 
 3->4 
 
 
 b 
 
<!-- 10 -->
 
 10 
 
 10 
 
<!-- 4->10 -->
 
 4->10 
 
 
 ε 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 5->6 -->
 
 5->6 
 
 
 b 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 6->7 -->
 
 6->7 
 
 
 ε 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 7->8 -->
 
 7->8 
 
 
 a 
 
<!-- 8->10 -->
 
 8->10 
 
 
 ε 
 
<!-- 9->1 -->
 
 9->1 
 
 
 ε 
 
<!-- 9->5 -->
 
 9->5 
 
 
 ε 
 
<!-- 10->12 -->
 
 10->12 
 
 
 ε 
 
<!-- 10->9 -->
 
 10->9 
 
 
 ε

After having constructed a non-deterministic <span style="font-variant:small-caps;">Fsm</span> for the regular expression,
we convert it into a deterministic <span style="font-variant:small-caps;">Fsm</span>.

In [6]:
const dfa = nfa2dfa(nfa);
console.log("Constructed DFA:");
console.log(dfa);

Constructed DFA:
{
  Q: Set(6) {
    '[1,11,12,5,9]',
    '[2,3]',
    '[6,7]',
    '[1,10,12,4,5,9]',
    '[1,10,12,5,8,9]',
    '∅'
  },
  Sigma: Set(2) { 'a', 'b' },
  delta: Map(12) {
    '[1,11,12,5,9],a' => '[2,3]',
    '[1,11,12,5,9],b' => '[6,7]',
    '[2,3],a' => '∅',
    '[2,3],b' => '[1,10,12,4,5,9]',
    '[6,7],a' => '[1,10,12,5,8,9]',
    '[6,7],b' => '∅',
    '[1,10,12,4,5,9],a' => '[2,3]',
    '[1,10,12,4,5,9],b' => '[6,7]',
    '[1,10,12,5,8,9],a' => '[2,3]',
    '[1,10,12,5,8,9],b' => '[6,7]',
    '∅,a' => '∅',
    '∅,b' => '∅'
  },
  q0: '[1,11,12,5,9]',
  F: Set(3) { '[1,11,12,5,9]', '[1,10,12,4,5,9]', '[1,10,12,5,8,9]' }
}


In [8]:
// Voraussetzung: viz.js-Factory "instance" ist verfügbar
// import { instance } from "@viz-js/viz";

async function showDFA(dfa: {
  Q: Set<string>;
  Sigma: Set<string>;
  delta: Map<string, string>;
  q0: string;
  F: Set<string>;
}) {
  // --- 1) Kanonische Repräsentation für Zustände ---
  const canon = (s: string) => {
    if (s === "∅") return "∅";
    try {
      const arr = JSON.parse(s);
      if (Array.isArray(arr)) {
        // numerisch sortieren, Leerzeichen egal
        return JSON.stringify(arr.map(Number).sort((a, b) => a - b));
      }
    } catch {}
    return (s ?? "").trim();
  };

  const pretty = (raw: string) => {
    if (raw === "∅") return "∅";
    try {
      const arr = JSON.parse(raw);
      if (Array.isArray(arr)) return `{${arr.join(", ")}}`;
    } catch {}
    return raw;
  };

  // --- 2) Alles normalisieren (Q, F, delta, q0) ---
  const Q = new Set([...dfa.Q].map(canon));
  const F = new Set([...dfa.F].map(canon));
  const q0 = canon(dfa.q0);

  const delta = new Map<string, string>();
  for (const [k, v] of dfa.delta.entries()) {
    const i = k.lastIndexOf(",");
    if (i < 0) continue;
    const from = canon(k.slice(0, i));
    const sym = k.slice(i + 1);
    delta.set(`${from},${sym}`, canon(v));
  }

  // --- 3) Zustände sortieren: Start zuerst, ∅ zuletzt, sonst lexikographisch ---
  const states = [...Q].sort((a, b) => {
    if (a === q0 && b !== q0) return -1;
    if (b === q0 && a !== q0) return 1;
    if (a === "∅" && b !== "∅") return 1;
    if (b === "∅" && a !== "∅") return -1;
    return a.localeCompare(b);
  });

  // stabile Namen S0, S1, …
  const nameOf = new Map<string, string>();
  states.forEach((q, i) => nameOf.set(q, `S${i}`));

  // --- 4) DOT bauen; pro Knoten shape explizit setzen ---
  const lines: string[] = [];
  lines.push("digraph DFA {");
  lines.push("  rankdir=LR;");
  lines.push('  __start [shape=point, label=\"\"];');
  lines.push(`  __start -> "${nameOf.get(q0)}";`);

  // Knoten (alle in EINER Schleife; shape je nach F)
  for (const q of states) {
    const id = nameOf.get(q)!;
    const label = `${id} ${pretty(q).replace(/"/g, '\\"')}`;
    const shape = F.has(q) ? "doublecircle" : "circle";
    lines.push(`  "${id}" [label="${label}", shape=${shape}];`);
  }

  // Kanten
  for (const [key, tgt] of delta.entries()) {
    const i = key.lastIndexOf(",");
    if (i < 0) continue;
    const from = key.slice(0, i);
    const sym = key.slice(i + 1);
    const fromId = nameOf.get(from);
    const toId = nameOf.get(tgt);
    if (!fromId || !toId) continue;
    lines.push(`  "${fromId}" -> "${toId}" [label="${sym}"];`);
  }

  lines.push("}");

  const dot = lines.join("\n");
  const viz = await instance();
  const svg = await viz.renderString(dot, { format: "svg" });

  if (typeof display !== "undefined" && (display as any).html) {
    (display as any).html(svg);
  } else {
    console.log(svg);
  }
}
await showDFA(dfa);

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.1 (20251006.0113)
 -->
<!-- Title: DFA Pages: 1 -->
 
 
 DFA 
 
<!-- __start -->
 
 __start 
 
 
<!-- S0 -->
 
 S0 
 
 
 S0 {1, 5, 9, 11, 12} 
 
<!-- __start->S0 -->
 
 __start->S0 
 
 
 
<!-- S3 -->
 
 S3 
 
 S3 {2, 3} 
 
<!-- S0->S3 -->
 
 S0->S3 
 
 
 a 
 
<!-- S4 -->
 
 S4 
 
 S4 {6, 7} 
 
<!-- S0->S4 -->
 
 S0->S4 
 
 
 b 
 
<!-- S1 -->
 
 S1 
 
 
 S1 {1, 4, 5, 9, 10, 12} 
 
<!-- S1->S3 -->
 
 S1->S3 
 
 
 a 
 
<!-- S1->S4 -->
 
 S1->S4 
 
 
 b 
 
<!-- S2 -->
 
 S2 
 
 
 S2 {1, 5, 8, 9, 10, 12} 
 
<!-- S2->S3 -->
 
 S2->S3 
 
 
 a 
 
<!-- S2->S4 -->
 
 S2->S4 
 
 
 b 
 
<!-- S3->S1 -->
 
 S3->S1 
 
 
 b 
 
<!-- S5 -->
 
 S5 
 
 S5 ∅ 
 
<!-- S3->S5 -->
 
 S3->S5 
 
 
 a 
 
<!-- S4->S2 -->
 
 S4->S2 
 
 
 a 
 
<!-- S4->S5 -->
 
 S4->S5 
 
 
 b 
 
<!-- S5->S5 -->
 
 S5->S5 
 
 
 a 
 
<!-- S5->S5 -->
 
 S5->S5 
 
 
 b

Note that the resulting DFA is not minimal, since we can identify some states.

In [ ]:
console.log("Akzeptierende Zustände:");
for (const f of dfa.F) console.log(f);
console.log("Alle Zustände:");
for (const q of dfa.Q) console.log(q);